In [1]:
import json
import prompt_rag
import torch
import os
import sys

os.environ['CUDA_VISIBLE_DEVICES'] = '1,2'

dirs = ["../.."]
for _dir in dirs:
    if _dir not in sys.path:
        sys.path.append(_dir)

from swift.llm import (
    ModelType, get_vllm_engine, get_default_template_type,
    get_template, inference_vllm, VllmGenerationConfig
)
from custom import CustomModelType

# model_type = CustomModelType.mixtral_moe_7b_instruct_gptq_int4
model_type = CustomModelType.llama_3_70b_instruct_awq
llm_engine = get_vllm_engine(
    model_type, 
    # torch_dtype=torch.float16,  # 检查正确的数据类型！！！！
    tensor_parallel_size=2,
    max_model_len=4096,
    # gpu_memory_utilization=0.95,
    # model_id_or_path="/home/css/models/Mixtral-8x7B-Instruct-v0.1-GPTQ-int4",
    engine_kwargs = {
        # "enforce_eager": True,
        "max_num_seqs": 64,
        "seed": 42,
    }
)

template_type = get_default_template_type(model_type)
template = get_template(template_type, llm_engine.hf_tokenizer)

generation_config = VllmGenerationConfig(
    max_new_tokens=2048,
    temperature=0,
)

get_resp_list = lambda request_list : inference_vllm(
    llm_engine, template, request_list, 
    generation_config=generation_config, 
    use_tqdm=True, 
)

search_engine = "brave"
model_name = 'llama3'
K = 5
sort = False

with open(f"/home/hanlv/workspace/data/machine_learning/dataset/research/misinformation_dataset/COVMIS-2024/train_{search_engine}_search.json", "r") as f:
    data_search = json.load(f)

# data_search = data_search[:10] + [data_search[9690]]
prompt_rag.update_train_search_llm(
    model_name, get_resp_list, search_engine, data_search,
    K=K, sort=sort
)


2024-06-14 04:58:38,136 - modelscope - INFO - PyTorch version 2.3.0 Found.
2024-06-14 04:58:38,138 - modelscope - INFO - Loading ast index from /home/hanlv/.cache/modelscope/ast_indexer
2024-06-14 04:58:38,172 - modelscope - INFO - Loading done! Current index file version is 1.15.0, with md5 21bc0b9ccf26bcf3f4ca2e675ec8875d and a total number of 980 components indexed
[INFO:swift] Successfully registered `/home/hanlv/workspace/code/research/infodemic/LLM/swift/swift/llm/data/dataset_info.json`
[INFO:swift] Loading the model using model_dir: /home/css/models/llama-3-70b-instruct-awq
[INFO:swift] Setting torch_dtype: torch.float16
[INFO:swift] model_config: LlamaConfig {
  "_name_or_path": "/home/css/models/llama-3-70b-instruct-awq",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 128000,
  "eos_token_id": 128001,
  "hidden_act": "silu",
  "hidden_size": 8192,
  "initializer_range": 0.02,
  "intermediate_size": 286

WARNING 06-14 04:58:39 config.py:216] awq quantization is not fully optimized yet. The speed can be slower than non-quantized models.


2024-06-14 04:58:41,212	INFO worker.py:1753 -- Started a local Ray instance.


INFO 06-14 04:58:42 config.py:623] Defaulting to use mp for distributed inference
INFO 06-14 04:58:42 llm_engine.py:161] Initializing an LLM engine (v0.5.0) with config: model='/home/css/models/llama-3-70b-instruct-awq', speculative_config=None, tokenizer='/home/css/models/llama-3-70b-instruct-awq', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.float16, max_seq_len=4096, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=2, disable_custom_all_reduce=True, quantization=awq, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), seed=42, served_model_name=/home/css/models/llama-3-70b-instruct-awq)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


...

WARNING 06-14 20:30:23 scheduler.py:1089] Sequence group 14294 is preempted by PreemptionMode.RECOMPUTE mode because there is not enough KV cache space. This can affect the end-to-end performance. Increase gpu_memory_utilization or tensor_parallel_size to provide more KV cache memory. total_num_cumulative_preemption=3301


(raylet) [2024-06-14 20:30:24,427 E 3332687 3332717] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-06-14_04-58-39_144518_3332349 is over 95% full, available space: 16366706688; capacity: 1967317549056. Object creation will fail if spilling is required.
100%|██████████| 14384/14384 [15:36:35<00:00,  3.91s/it]
(raylet) [2024-06-14 20:36:44,844 E 3332687 3332717] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-06-14_04-58-39_144518_3332349 is over 95% full, available space: 16369074176; capacity: 1967317549056. Object creation will fail if spilling is required.
(raylet) [2024-06-14 20:36:54,850 E 3332687 3332717] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-06-14_04-58-39_144518_3332349 is over 95% full, available space: 16369065984; capacity: 1967317549056. Object creation will fail if spilling is required.


(raylet) [2024-06-14 20:37:04,858 E 3332687 3332717] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-06-14_04-58-39_144518_3332349 is over 95% full, available space: 16312844288; capacity: 1967317549056. Object creation will fail if spilling is required.
(raylet) [2024-06-14 20:37:14,870 E 3332687 3332717] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-06-14_04-58-39_144518_3332349 is over 95% full, available space: 16312827904; capacity: 1967317549056. Object creation will fail if spilling is required.
(raylet) [2024-06-14 20:37:24,882 E 3332687 3332717] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-06-14_04-58-39_144518_3332349 is over 95% full, available space: 16312823808; capacity: 1967317549056. Object creation will fail if spilling is required.
(raylet) [2024-06-14 20:37:34,895 E 3332687 3332717] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-06-14_04-58-39_144518_3332349 is over 95% full, available space: 16312819712; capacity: 19

(VllmWorkerProcess pid=3336290) ERROR 06-14 21:06:39 multiproc_worker_utils.py:226] Exception in worker VllmWorkerProcess while processing method start_worker_execution_loop: [../third_party/gloo/gloo/transport/tcp/unbound_buffer.cc:81] Timed out waiting 1800000ms for recv operation to complete, Traceback (most recent call last):
(VllmWorkerProcess pid=3336290) ERROR 06-14 21:06:39 multiproc_worker_utils.py:226]   File "/home/hanlv/miniconda3/envs/swift/lib/python3.11/site-packages/vllm/executor/multiproc_worker_utils.py", line 223, in _run_worker_process
(VllmWorkerProcess pid=3336290) ERROR 06-14 21:06:39 multiproc_worker_utils.py:226]     output = executor(*args, **kwargs)
(VllmWorkerProcess pid=3336290) ERROR 06-14 21:06:39 multiproc_worker_utils.py:226]              ^^^^^^^^^^^^^^^^^^^^^^^^^
(VllmWorkerProcess pid=3336290) ERROR 06-14 21:06:39 multiproc_worker_utils.py:226]   File "/home/hanlv/miniconda3/envs/swift/lib/python3.11/site-packages/torch/utils/_contextlib.py", line 115

(raylet) [2024-06-14 21:06:46,932 E 3332687 3332717] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-06-14_04-58-39_144518_3332349 is over 95% full, available space: 16299278336; capacity: 1967317549056. Object creation will fail if spilling is required.
(raylet) [2024-06-14 21:06:56,945 E 3332687 3332717] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-06-14_04-58-39_144518_3332349 is over 95% full, available space: 16299274240; capacity: 1967317549056. Object creation will fail if spilling is required.
(raylet) [2024-06-14 21:07:06,958 E 3332687 3332717] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-06-14_04-58-39_144518_3332349 is over 95% full, available space: 16299270144; capacity: 1967317549056. Object creation will fail if spilling is required.
(raylet) [2024-06-14 21:07:16,969 E 3332687 3332717] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-06-14_04-58-39_144518_3332349 is over 95% full, available space: 16299266048; capacity: 19